In [1]:
import numpy as np
#import scipy.misc
#import matplotlib
#import matplotlib.pyplot as plt
#%matplotlib inline

import math
import random

#from random import shuffle

In [2]:
X = [1,2,3,4,5]
random.shuffle(X)
X

[2, 1, 3, 4, 5]

In [3]:
numNetworks = 100
numNodesPerNetwork = 100
inputFeatures = 10
outputFeatures = 5

globalInnovationNumber = 0

assert inputFeatures < numNodesPerNetwork
assert outputFeatures < numNodesPerNetwork

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def clamp(x, a, b):
    return min(max(a, x), b)

class Node:
    """
    This represents a neural network node with an integer label nodeId, 
    and a special type, one of ["input", "output", "hidden"]
    """
    def __init__(self, nodeId, nodeType):
        self.nodeId = nodeId
        assert nodeType == "input" or nodeType == "output" or nodeType == "hidden"
        self.nodeType = nodeType
        
    def copy(self):
        return Node(self.nodeId, self.nodeType)    
        
    def __str__(self):
        return "Node: " + str(self.nodeId) + ", " + self.nodeType
    
        
class Edge:
    """
    This class represent a connection (axon) in a neuroevolution network with its data.
    nodeIn and nodeOut are the ids of the input and output nodes,
    enabled is a boolean representing its ability to propogate into future neural networks,
    innovId is the labeled innovation number which corresponds to the same edge in other NNs,
    and weight is the normal edge weight.
    """
    def __init__(self, nodeIn, nodeOut, enabled, innovId, weight):
        self.nodeInId = nodeIn
        self.nodeOutId = nodeOut
        self.enabled = enabled
        self.innovId = innovId
        self.weight = weight
        
    def copy(self):
        return Edge(self.nodeInId, self.nodeOutId, self.enabled, self.innovId, self.weight)
        
    def __str__(self):
        return "Edge: " + str(self.nodeInId) + " -> " + str(self.nodeOutId) + ", Enabled: " + \
            str(self.enabled) + ", InnovID: " + str(self.innovId) + ", Weight: " + str(self.weight)
    
class NeuroevolutionNetwork:
    
    # self.nodes is a dictionary indexed by (node id -> node object)
    # self.edges is indexed by (in node id -> edge object, from in node to out node)
    def __init__(self, numNodes, numInput, numOutput):
        assert numInput + numOutput < numNodes
        
        self.nodes = dict() 
        self.edges = dict()
        self.listEdges = []
        self.numNodesCounter = 0
        
        self.numInput = numInput
        self.numOutput = numOutput
        self.initNodes = numNodes
        
        for _ in range(numInput):
            self.addNode("input")
        for _ in range(numOutput):
            self.addNode("output")
        for _ in range(numNodes - numInput - numOutput):
            self.addNode("hidden")
        
    # Special copy method to handle it ourselves, so it's not just a shallow/deep clone
    def copy(self):
        copyNetwork = NeuroevolutionNetwork(self.initNodes, self.numInput, self.numOutput)
        
        for nodeId, node in self.nodes.items():
            # For every node id, copy its node to the new network
            copyNetwork.nodes[nodeId] = node.copy()
            
        for nodeId, edgeList in self.edges.items():
            # For every node in id, copy the list of outgoing edges to the new network
            copyEdgeArr = []
            for edge in edgeList:
                copyEdgeArr.append(edge.copy())
            copyNetwork.edges[nodeId] = copyEdgeArr  
            
        for pair in self.listEdges:
            copyNetwork.listEdges.append(tuple(pair))
            
        return copyNetwork
    
    def printAllEdges(self):
        for nodeInId, nodeArr in self.edges.items():
            print(nodeInId)
            for node in nodeArr:
                print(str(node))
                
    def numNodes(self):
        return len(self.nodes)
    
    def numEdges(self):
        return len(self.listEdges)
        
    def addNode(self, nodeType):
        nodeId = self.numNodesCounter
        self.nodes[nodeId] = Node(nodeId, nodeType)
        self.numNodesCounter += 1
        return nodeId
        
    def hasNode(self, nodeId):
        return nodeId in self.nodes    
        
    # Return true if a connection exists between nodes with these ids,
    # a connection from in -> out
    def isConnected(self, nodeInId, nodeOutId):    
        """if hasNode(nodeInId) and hasNode(nodeOutId):
            if nodeInId in self.edges:
                connectionsOut = self.edges[nodeInId]
                for nodeOut in connectionsOut:
                    if nodeOut.nodeId == nodeOutId:
                        return True
        return False"""
        return (nodeInId, nodeOutId) in self.listEdges
    
    # Return true if there is a valid connection from in -> out
    # Invariant to whether or not the nodes are connected or not
    def allowedConnection(self, nodeInId, nodeOutId):
        if self.hasNode(nodeInId) and self.hasNode(nodeOutId):
            inNode = self.nodes[nodeInId]
            outNode = self.nodes[nodeOutId]
            doubleInput = inNode.nodeType == "input" and outNode.nodeType == "output"
            doubleOutput = inNode.nodeType == "input" and outNode.nodeType == "output"
            return not doubleInput and not doubleOutput
        else:
            return False
        
    # Variant to whether nodes are connected. To be able to add a new connection, the nodes
    # must follow these conditions:
    """
    Nodes exist
    Nodes are not connected in either direction
    The end node is not an input
    The start node is not an output
    New: the new edge cannot create a cycle
    """
    def canAddConnection(self, nodeInId, nodeOutId):
        assert nodeInId != nodeOutId
        if not self.allowedConnection(nodeInId, nodeOutId):
            return False
        inToOutC = self.isConnected(nodeInId, nodeOutId)
        outToInC = self.isConnected(nodeOutId, nodeInId)
        endInput = self.nodes[nodeOutId].nodeType == "input"
        startOutput = self.nodes[nodeInId].nodeType == "output"
        longPathExists = self.findPathBetween(nodeOutId, nodeInId) # Check for possible cycles
        return not (inToOutC or outToInC or endInput or startOutput or longPathExists)
        
    # Select a random pair of nodes, that have no connection either way,
    # and are not both input and not both output,
    # and add a connection between them, from in -> out.
    def mutateAddConnection(self):
        assert len(self.nodes) >= 2
        
        # Generate a shuffled list of possible future connections
        availIds = [node.nodeId for _,node in self.nodes.items()]
        lenIds = len(availIds)
        availablePairs = [(availIds[i], availIds[j]) for i in range(lenIds) for j in range(lenIds)]
        random.shuffle(availablePairs)
        
        # Find two nodes that have no connection
        # Once found, add the one new connection, and return
        while True: 
            if len(availablePairs) == 0: # No more possible connections to check
                return
            pair = availablePairs.pop() 
            if pair[0] == pair[1]:
                continue
            # Select a random connection, that is neither a self connection (does not exist),
            # nor an existing connection in either direction.
            # inToOutC = self.isConnected(pair[0], pair[1])
            # outToInC = self.isConnected(pair[1], pair[0])
            if self.canAddConnection(pair[0], pair[1]):
                global globalInnovationNumber
                self.addEdgeWithInnovId(pair[0], pair[1], True, globalInnovationNumber, edgeWeight=random.random())
                globalInnovationNumber += 1
                return
            
    def mutateRemoveConnection(self):
        # Find a random edge to mutate, assuming one exists
        assert len(self.edges) > 0
        
        nodeInId, nodeOutId = None, None
        candidateEdges = [i for i in range(len(self.listEdges))]
        random.shuffle(candidateEdges)
        while True:
            if len(candidateEdges) == 0:
                return
            randomEdgeIndex = candidateEdges.pop(0)
             
            oldEdge = tuple(self.listEdges[randomEdgeIndex])
            nodeInId, nodeOutId = oldEdge[0], oldEdge[1]
            if self.allowedConnection(nodeInId, nodeOutId):
                break
            else:
                continue

        self.removeEdge(nodeInId, nodeOutId)
            
    """
    This mutates the graph by finding a random existing edge,
    and then splitting into two edges with a new connection in the middle,
    as per Stanley & Miikkulainen.
    """    
    def mutateSplitConnection(self):
        # Find a random edge to mutate, assuming one exists
        assert len(self.edges) > 0
        
        nodeInId, nodeOutId = None, None
        candidateEdges = [i for i in range(len(self.listEdges))]
        random.shuffle(candidateEdges)
        edgeObj = None
        while True:
            if len(candidateEdges) == 0:
                return
            randomEdgeIndex = candidateEdges.pop(0)
             
            oldEdge = tuple(self.listEdges[randomEdgeIndex])
            nodeInId, nodeOutId = oldEdge[0], oldEdge[1]
            
            edgeObj = self.findEdge(nodeInId, nodeOutId)
            if not edgeObj.enabled:
                continue
            
            if self.canAddConnection(nodeInId, nodeOutId):
                break
            else:
                continue
        
        # TEMP:
        # self.removeEdge(nodeInId, nodeOutId)
        # No, do not remove the edge. It must be preserved to play a part in later
        # NN "reproduction" and similarity measures

        edgeObj.enabled = False
        
        newMiddleNode = self.addNode("hidden")
        
        global globalInnovationNumber
        self.addEdgeWithInnovId(nodeInId, newMiddleNode, True, globalInnovationNumber, edgeWeight=random.random())
        globalInnovationNumber += 1
        self.addEdgeWithInnovId(newMiddleNode, nodeOutId, True, globalInnovationNumber, edgeWeight=random.random())
        globalInnovationNumber += 1
            
        
    def mutateChangeWeight(self):        
        # Find a random edge to mutate, assuming one exists
        assert len(self.edges) > 0
        
        randomEdgeIndex = int(len(self.listEdges) * random.random())
        oldEdge = tuple(self.listEdges[randomEdgeIndex])
        nodeInId, nodeOutId = oldEdge[0], oldEdge[1]
        edgeObj = self.findEdge(nodeInId, nodeOutId)
        
        weight = edgeObj.weight
        if random.random() < 0.5:
            weight -= 0.1
        else:
            weight += 0.1
        weight = clamp(weight, 0, 1)
        
        edgeObj.weight = weight
    
    
    def findEdge(self, nodeInId, nodeOutId):
        for nodeOutIndex in range(len(self.edges[nodeInId])):
            possibleEdge = self.edges[nodeInId][nodeOutIndex]
            if possibleEdge.nodeOutId == nodeOutId:
                return possibleEdge
        return None
        
    def removeEdge(self, nodeInId, nodeOutId):
        assert nodeInId in self.edges
        assert (nodeInId, nodeOutId) in self.listEdges
        
        self.listEdges.remove((nodeInId, nodeOutId))
        
        # Look for the correct node within the outgoing connections
        # i.e. find the connection in -> out
        for nodeOutIndex in range(len(self.edges[nodeInId])):
            nodeOut = self.edges[nodeInId][nodeOutIndex]
            if nodeOut.nodeOutId == nodeOutId:
                self.edges[nodeInId].pop(nodeOutIndex)
                return  
            
    # Simple tree traversal, to check for a path between in -> out, of any length
    def findPathBetween(self, nodeInId, nodeOutId):
        fringe = [nodeInId]
        marked = set()
        while len(fringe) > 0:
            v = fringe.pop(0)
            if v == nodeOutId: # This case is if nodeInId == nodeOutId
                return True
            marked.add(v)
            if v not in self.edges: # No edges starting from v
                continue
            for edge in self.edges[v]:
                w = edge.nodeOutId
                if w == nodeOutId:
                    return True
                if w not in marked:
                    fringe.append(w)
        return False
            
    def addEdgeWithInnovId(self, nodeInId, nodeOutId, enabled, innovId, edgeWeight):
        assert self.canAddConnection(nodeInId, nodeOutId)
        
        newEdge = Edge(nodeInId, nodeOutId, enabled, innovId, edgeWeight)
        
        self.listEdges.append((nodeInId, nodeOutId))
        if nodeInId not in self.edges:
            self.edges[nodeInId] = []
        self.edges[nodeInId].append(newEdge)
            
        
    """
    Get all of the directed edges of the NN, but in reverse i.e. incoming edges,
    also representing an in-degree count indexed by vertex.
    """
    def getReversedEdges(self):
        reversedEdges = dict()
        
        for edgeInId, edgeList in self.edges.items():
            for edge in edgeList:
                if edge.nodeOutId not in reversedEdges:
                    reversedEdges[edge.nodeOutId] = []
                reversedEdges[edge.nodeOutId].append(edge.nodeInId)
        
        return reversedEdges
    
    """
    The new forwarding algorithm works as follows, in a DAG:
    obtain the edges of the graph, reversed;
    determine the in-degree of each vertex, using above;
    use Kahn's algorithm -> remove vertices of in-degree zero,
    repeat until end or a valid topological sort has been found;
    update vertices in topological order
    
    Proof that Kahn's algorithm terminates in a valid topo sort <-> graph is a DAG
    is left as an exercise for the reader.
    """
    
    """
    Return a list of the nodes 
    that represent a valid topological sort,
    starting with input edges (guaranteed to be in-degree zero).
    """
    def graphTopoSort(self):
        reversedEdges = self.getReversedEdges()
        
        marked = dict()
        topoSort = []
        
        for nodeId,_ in self.nodes.items():
            marked[nodeId] = False
        
        while len(topoSort) < len(self.nodes):
            degZeroIncomingIds = []
            
            for nodeId,_ in self.nodes.items():
                # Boolean short circ. V
                # The second case can only come about when connections are discarded
                if (nodeId not in reversedEdges or len(reversedEdges[nodeId]) == 0) and not marked[nodeId]: 
                    # nodeId must have an in-degree zero
                    degZeroIncomingIds.append(nodeId)
                    marked[nodeId] = True
                    
                    # No outgoing edges, we're done
                    if nodeId not in self.edges:
                        continue
                    
                    # Discard all outgoing connections
                    outgoingEdges = self.edges[nodeId]
                    
                    for edge in outgoingEdges:
                        if edge.nodeOutId in reversedEdges:
                            reversedEdges[edge.nodeOutId].remove(nodeId)
            
            topoSort = topoSort + degZeroIncomingIds
        return topoSort
    
    """
    Calculate the entire NN output, starting from the input.
    Again, the topological ordering is used to guarantee the nodes are calculated correctly
    i.e. all incoming edges have valid values.
    """
    def forwardStep(self, inputData):
        inputData = np.array(inputData)
        assert inputData.shape[0] == self.numInput
        
        # nodeData is the final result of the NN, indexed by nodeId
        # Initialize the input, which is ideally of shape (input nodes X 1)
        nodeData = dict()
        for i in range(len(inputData)):
            nodeData[i] = inputData[i]
        
        # Update the nodes in topo order, using the incoming edges to determine weights and values
        reversedEdges = self.getReversedEdges()
        topoSort = self.graphTopoSort()
        
        for nodeId in topoSort:
            nodeType = self.nodes[nodeId].nodeType
            
            # Input nodes must not be overridden. Their degree is technically zero
            if nodeType == 'input':
                continue
            
            # Find the list of incoming edges
            incomingEdges = []
            if nodeId in reversedEdges:
                incomingEdges = reversedEdges[nodeId]
            
            result = 0
            if len(incomingEdges) == 0:
                result = 0.5
            else:
                nodeSum = 0
                # Sum up the incoming values, and take the activation function of the sum,
                # while noting that only enabled edges count in the calculation.
                for inVertex in incomingEdges:
                    inEdge = self.findEdge(inVertex, nodeId)
                    if inEdge.enabled:  
                        value = nodeData[inVertex]
                        weight = inEdge.weight
                        nodeSum += value * weight
                result = sigmoid(nodeSum)
                
            nodeData[nodeId] = result
        
        return nodeData
    
    def score(self, currentNodeData, desiredOutput, lossFunction):
        predictedOutput = np.zeros((self.numOutput))
        for i in range(0, self.numOutput):
            predictedOutput[i] = currentNodeData[self.numInput + i]
        assert predictedOutput.shape == desiredOutput.shape
        loss = lossFunction(predictedOutput, desiredOutput)
        return predictedOutput, desiredOutput, loss
        
    """def forwardStep(self, inputData):
        inputData = np.array(inputData)
        assert inputData.shape[0] == self.numInput
        
        for i in range(len(inputData)):
            nodeData[i] = inputData[i]
            
        # Use Dijkstra's algorithm to update the NN as we go, 
        # using the input data as the starting fringe.
        
        # Ideally, the NN is a DAG, but this will work even if not.
        
        nodeData = dict()
        prev = dict()
        marked = dict()
        
        queue = [i for i in range(len(self.numInput))]
        
        for nodeId, _ in self.nodes.items():
            nodeData[nodeId] = -9999
            prev[nodeId] = -1
            marked[nodeId] = False
        
        while len(queue) > 0:
            v = queue.pop() # A node id
            marked[v] = True
            for edge in self.edges[v]:
                w = edge.nodeOutId
                if :
                    nodeData[]"""

In [4]:
# Unit tests for neuroevolution network

testNet = NeuroevolutionNetwork(numNodesPerNetwork, inputFeatures, outputFeatures)

# Nodes are initialized correctly
"""
for k,v in testNet.nodes.items():
    print(v)
"""
    
# Edges are initialized correctly
print("-----------------------------------")
print("Test #2")
print(testNet.edges, testNet.listEdges)

print("-----------------------------------")
print("Test #3")
for _ in range(5):
    testNet.mutateAddConnection()

testNet.printAllEdges()
    
print(testNet.listEdges)

print("-----------------------------------")
print("Test #4")

testEdge = testNet.listEdges[0]
testNet.removeEdge(testEdge[0], testEdge[1])

testNet.printAllEdges()

print(testNet.listEdges)

print("-----------------------------------")
print("Test #5")

testNet.mutateSplitConnection()

testNet.printAllEdges()

print(testNet.listEdges)

print("-----------------------------------")
print("Test #5.5")

for _ in range(10):
    testNet.mutateChangeWeight()

testNet.printAllEdges()

print("-----------------------------------")
print("Test #6")

testNetCopy = testNet.copy()
testNetCopy.mutateAddConnection()

print("Original network: ")
testNet.printAllEdges()
print("Copy network: ")
testNetCopy.printAllEdges()

-----------------------------------
Test #2
{} []
-----------------------------------
Test #3
68
Edge: 68 -> 42, Enabled: True, InnovID: 0, Weight: 0.18544405192789204
42
Edge: 42 -> 21, Enabled: True, InnovID: 1, Weight: 0.7197288840128838
34
Edge: 34 -> 51, Enabled: True, InnovID: 2, Weight: 0.8620649479138409
23
Edge: 23 -> 87, Enabled: True, InnovID: 3, Weight: 0.9308895888251557
64
Edge: 64 -> 85, Enabled: True, InnovID: 4, Weight: 0.8352194351401537
[(68, 42), (42, 21), (34, 51), (23, 87), (64, 85)]
-----------------------------------
Test #4
68
42
Edge: 42 -> 21, Enabled: True, InnovID: 1, Weight: 0.7197288840128838
34
Edge: 34 -> 51, Enabled: True, InnovID: 2, Weight: 0.8620649479138409
23
Edge: 23 -> 87, Enabled: True, InnovID: 3, Weight: 0.9308895888251557
64
Edge: 64 -> 85, Enabled: True, InnovID: 4, Weight: 0.8352194351401537
[(42, 21), (34, 51), (23, 87), (64, 85)]
-----------------------------------
Test #5
68
42
Edge: 42 -> 21, Enabled: True, InnovID: 1, Weight: 0.719728

In [5]:
testNode = Node(85, 'hidden')
print(testNode.copy())

Node: 85, hidden


In [6]:
"""
Calculate the distance delta between two neuroevolution networks.
This heuristic is important in determining a normalized fitness score,
which rewards successful organisms but ensures that one species does not take over.

In Stanley, Miikkulainen, this is defined as

delta = c_1 * E / N + c_2 * D / N + c_3 * W
where E is the number of excess genes,
D is the number of disjoint genes, 
N is the normalization factor (number of genes total),
and W is the average weight distance between shared genes.

Ideally, the weights have constraint 0 <= c_1, c_2, c_3 <= 1, 
and should also be consistent.
"""
def neuroNetworkDiff(nn1, nn2, c1, c2, c3):
    # Calculate excess number of genes by count
    excess = abs(nn1.numEdges() - nn2.numEdges())
    
    # Disjoint genes just need to be counted,
    # shared genes must have their weight differences averaged
    disjoint = 0
    sharedGenesDiff = []
    
    numGenes = max(nn1.numEdges(), nn2.numEdges())
    
    firstEdgeInnovIds = set()
    weightsByInnovId = dict() # Guaranteed to be surjective, stores weights of the first NN
    
    for _,outEdges in nn1.edges.items():
        for edge in outEdges:
            firstEdgeInnovIds.add(edge.innovId)
            weightsByInnovId[edge.innovId] = edge.weight
    
    for _,outEdges in nn2.edges.items():
        for secondEdge in outEdges:
            if secondEdge.innovId in firstEdgeInnovIds:
                firstEdgeWeight = weightsByInnovId[secondEdge.innovId]
                secondEdgeWeight = secondEdge.weight
                # secondEdge = nn2.edges[node.nodeId]
                sharedGenesDiff.append(abs(firstEdgeWeight - secondEdgeWeight))
            else:
                disjoint += 1

    avgDiffW = np.sum(sharedGenesDiff) / len(sharedGenesDiff)
    
    # print(excess, disjoint, numGenes, avgDiffW)
    
    return c1 * excess / numGenes + \
        c2 * disjoint / numGenes + \
        c3 * avgDiffW
        

def crossNeuralNetworks(nn1, nn2):
    assert nn1.numInput == nn2.numInput
    assert nn1.numOutput == nn2.numOutput
    newNumNodes = max(nn1.numNodes(), nn2.numNodes())
    
    crossedNet = NeuroevolutionNetwork(newNumNodes, nn1.numInput, nn1.numOutput)
    
    firstEdgeInnovIds = set()
    secondEdgeInnovIds = set()
    firstEdgesByInnovId = dict() # Guaranteed to be surjective, stores weights of the first NN
    secondEdgesByInnovId = dict()
    enabledByInnovId = dict()
    
    for _,outEdges in nn1.edges.items():
        for edge in outEdges:
            firstEdgeInnovIds.add(edge.innovId)
            firstEdgesByInnovId[edge.innovId] = edge
            enabledByInnovId[edge.innovId] = edge.enabled
    
    for _,outEdges in nn2.edges.items():
        for edge in outEdges:
            secondEdgeInnovIds.add(edge.innovId)
            secondEdgesByInnovId[edge.innovId] = edge
            if edge.innovId in enabledByInnovId:
                enabledByInnovId[edge.innovId] = enabledByInnovId[edge.innovId] and edge.enabled
            else:
                enabledByInnovId[edge.innovId] = edge.enabled
    
    """
    As per Stanley and Miikkulainen, merge the edges together by their innovation ids.
    Disjoint/excess innovations transfer fully, shared innovations must merge,
    weight by average, enabled by AND
    """
    
    sharedInnov = firstEdgeInnovIds.intersection(secondEdgeInnovIds)
    onlyFirstInnov = firstEdgeInnovIds.difference(secondEdgeInnovIds)
    onlySecondInnov = secondEdgeInnovIds.difference(firstEdgeInnovIds)
    
    #nodeIn, nodeOut, enabled, innovId, weight)
    for innovId in onlyFirstInnov:
        baseEdge = firstEdgesByInnovId[innovId]
        inId = baseEdge.nodeInId
        outId = baseEdge.nodeOutId
        if crossedNet.canAddConnection(inId, outId):
            crossedNet.addEdgeWithInnovId(inId, outId, enabledByInnovId[innovId], innovId, baseEdge.weight)
        
    for innovId in onlySecondInnov:
        baseEdge = secondEdgesByInnovId[innovId]
        inId = baseEdge.nodeInId
        outId = baseEdge.nodeOutId
        if crossedNet.canAddConnection(inId, outId):
            crossedNet.addEdgeWithInnovId(inId, outId, enabledByInnovId[innovId], innovId, baseEdge.weight)
            
    for innovId in sharedInnov:
        baseEdge = firstEdgesByInnovId[innovId]
        inId = baseEdge.nodeInId
        outId = baseEdge.nodeOutId
        
        firstWeight = firstEdgesByInnovId[innovId].weight
        secondWeight = secondEdgesByInnovId[innovId].weight
        avgWeight = 0.5 * (firstWeight + secondWeight)
        
        if crossedNet.canAddConnection(inId, outId):
            crossedNet.addEdgeWithInnovId(inId, outId, enabledByInnovId[innovId], innovId, avgWeight)
    
    return crossedNet
                

def squaredLoss(predicted, desired):
    return np.sum((predicted - desired) ** 2)    



In [7]:
print("-----------------------------------")
print("Test #7")

testNet = NeuroevolutionNetwork(numNodesPerNetwork, inputFeatures, outputFeatures)
for _ in range(10):
    testNet.mutateAddConnection()
    
testNetMutate = testNet.copy()

for _ in range(2):
    testNetMutate.mutateAddConnection()
for _ in range(5):
    testNetMutate.mutateSplitConnection()
    
testNet.printAllEdges()
testNetMutate.printAllEdges()

print(neuroNetworkDiff(testNet, testNetMutate, 0.5, 0.25, 0.2))

print("-----------------------------------")
print("Test #8")

testEdges = testNet.getReversedEdges()
for nodeId,listEdges in testEdges.items():
    print(nodeId)
    print(listEdges)

print("-----------------------------------")
print("Test #9")

testNet.printAllEdges()
topoSort = testNet.graphTopoSort()
print(topoSort)

-----------------------------------
Test #7
73
Edge: 73 -> 47, Enabled: True, InnovID: 6, Weight: 0.6517066994379094
79
Edge: 79 -> 57, Enabled: True, InnovID: 7, Weight: 0.43291584810673567
6
Edge: 6 -> 57, Enabled: True, InnovID: 8, Weight: 0.1395448361169339
63
Edge: 63 -> 92, Enabled: True, InnovID: 9, Weight: 0.29764532103306807
53
Edge: 53 -> 88, Enabled: True, InnovID: 10, Weight: 0.3060528568920675
99
Edge: 99 -> 69, Enabled: True, InnovID: 11, Weight: 0.9273158153645266
Edge: 99 -> 78, Enabled: True, InnovID: 14, Weight: 0.28938816253550825
18
Edge: 18 -> 10, Enabled: True, InnovID: 12, Weight: 0.5902348408116856
31
Edge: 31 -> 55, Enabled: True, InnovID: 13, Weight: 0.9906036663745842
93
Edge: 93 -> 68, Enabled: True, InnovID: 15, Weight: 0.8167024934234031
73
Edge: 73 -> 47, Enabled: True, InnovID: 6, Weight: 0.6517066994379094
79
Edge: 79 -> 57, Enabled: True, InnovID: 7, Weight: 0.43291584810673567
6
Edge: 6 -> 57, Enabled: True, InnovID: 8, Weight: 0.1395448361169339
63
E

In [8]:
print("-----------------------------------")
print("Test #10")

testNet = NeuroevolutionNetwork(15, 5, 5)
for _ in range(20):
    testNet.mutateAddConnection()
for _ in range(3):
    testNet.mutateSplitConnection()
for _ in range(20):
    testNet.mutateAddConnection()
    
testNet.printAllEdges()

-----------------------------------
Test #10
2
Edge: 2 -> 11, Enabled: True, InnovID: 18, Weight: 0.22602914527845452
Edge: 2 -> 10, Enabled: True, InnovID: 19, Weight: 0.6077859154686776
Edge: 2 -> 13, Enabled: True, InnovID: 43, Weight: 0.33147097022512173
11
Edge: 11 -> 8, Enabled: True, InnovID: 20, Weight: 0.4289665824070398
Edge: 11 -> 13, Enabled: True, InnovID: 29, Weight: 0.06565299864067753
Edge: 11 -> 7, Enabled: True, InnovID: 40, Weight: 0.627799201870831
Edge: 11 -> 6, Enabled: True, InnovID: 44, Weight: 0.5761200583020808
Edge: 11 -> 5, Enabled: True, InnovID: 49, Weight: 0.7091221158765454
0
Edge: 0 -> 11, Enabled: True, InnovID: 21, Weight: 0.35719572407702316
Edge: 0 -> 10, Enabled: True, InnovID: 27, Weight: 0.008276746033863969
Edge: 0 -> 12, Enabled: True, InnovID: 38, Weight: 0.725927647237748
Edge: 0 -> 13, Enabled: True, InnovID: 56, Weight: 0.49793139626128313
10
Edge: 10 -> 9, Enabled: True, InnovID: 22, Weight: 0.008672638459592541
Edge: 10 -> 6, Enabled: Tru

In [9]:
testNet.forwardStep([1.0, 0.0, 1.0, 0.0, 1.0])
# [16, 10, 4, 1, 97]
# -> [4,3.2,2,1,9.7]

{0: 1.0,
 1: 0.0,
 2: 1.0,
 3: 0.0,
 4: 1.0,
 5: 0.76989991963251603,
 6: 0.74201048011446291,
 7: 0.77686570461257598,
 8: 0.81554723709399735,
 9: 0.7464936293773019,
 10: 0.88912794734549672,
 11: 0.93340965943986787,
 12: 0.70644700166174412,
 13: 0.90904695751242237,
 14: 0.50710668503430512}

In [10]:
print("-----------------------------------")
print("Test #11")

# score(self, currentNodeData, desiredOutput, lossFunction):

inputData = np.array([1.0, 0.0, 1.0, 0.0, 1.0])
totalNodeData = testNet.forwardStep(inputData)
desired = np.array([0.0, 1.0, 0.0, 1.0, 0.0])

predicted, _, testLoss = testNet.score(totalNodeData, desired, squaredLoss)
print(predicted)
print(desired)
print(testLoss)

-----------------------------------
Test #11
[ 0.76989992  0.74201048  0.7768657   0.81554724  0.74649363]
[ 0.  1.  0.  1.  0.]
1.85410036207


In [11]:
print("-----------------------------------")
print("Test #12")

testNet = NeuroevolutionNetwork(15, 5, 5)

for _ in range(5):
    testNet.mutateAddConnection()
    
testNetMutate1 = testNet.copy()
testNetMutate2 = testNet.copy()

for _ in range(5):
    testNetMutate1.mutateAddConnection()
for _ in range(2):
    testNetMutate1.mutateSplitConnection()

for _ in range(5):
    testNetMutate2.mutateAddConnection()
for _ in range(2):
    testNetMutate2.mutateSplitConnection()
    
print("**************************")
testNet.printAllEdges()
print("**************************")
testNetMutate1.printAllEdges()
print("**************************")
testNetMutate2.printAllEdges()
print("**************************")

crossedNet = crossNeuralNetworks(testNetMutate1, testNetMutate2)
crossedNet.printAllEdges()

-----------------------------------
Test #12
**************************
10
Edge: 10 -> 11, Enabled: True, InnovID: 58, Weight: 0.8079969265986626
1
Edge: 1 -> 13, Enabled: True, InnovID: 59, Weight: 0.7280551635999067
Edge: 1 -> 14, Enabled: True, InnovID: 61, Weight: 0.3430627593066833
13
Edge: 13 -> 7, Enabled: True, InnovID: 60, Weight: 0.31333561451974157
0
Edge: 0 -> 11, Enabled: True, InnovID: 62, Weight: 0.13239242892587788
**************************
10
Edge: 10 -> 11, Enabled: True, InnovID: 58, Weight: 0.8079969265986626
Edge: 10 -> 14, Enabled: True, InnovID: 63, Weight: 0.4608329239991351
Edge: 10 -> 6, Enabled: True, InnovID: 64, Weight: 0.7114640426419299
1
Edge: 1 -> 13, Enabled: True, InnovID: 59, Weight: 0.7280551635999067
Edge: 1 -> 14, Enabled: True, InnovID: 61, Weight: 0.3430627593066833
13
Edge: 13 -> 7, Enabled: True, InnovID: 60, Weight: 0.31333561451974157
Edge: 13 -> 6, Enabled: True, InnovID: 67, Weight: 0.6123570593874937
0
Edge: 0 -> 11, Enabled: True, Innov

In [12]:
"""
Initialize a world class, which contains multiple neural networks and functions to calculate on them 
and evolve them towards a goal.
"""

numNetworks = 100
numTotalNodes = 15
numInput = 5
numOutput = 5

baseMutations = 0
initialMutations = 5
iterAddMutations = 3
iterSplitMutations = 0
iterRemoveMutations = 3
iterWeightMutations = 3

mutateProportion = 0.5

similarityThreshold = 0.35 # A distance less than this number implies two NNs are the same "species"
similarityPenalty = 0.25 # In Stanley, Miikkulainen, this number is 1, representing dividing by raw count

class World:
    
    def __init__(self, numNetworks, numTotalNodes, numInput, numOutput):
        self.networks = []
        
        baseNet = NeuroevolutionNetwork(numTotalNodes, numInput, numOutput)
        for _ in range(baseMutations):
            baseNet.mutateAddConnection()
            
        for _ in range(numNetworks - 1):
            baseNetMutate = baseNet.copy()
            for _ in range(initialMutations):
                baseNetMutate.mutateAddConnection()
            self.networks.append(baseNetMutate)
        
    def calcDiffNetMatrix(self):
        # Recommended values: 
        c1 = 0.5
        c2 = 0.25
        c3 = 0.2
        
        n = len(self.networks)
        results = np.zeros((n, n))
        for i in range(n):
            for j in range(n):
                if i < j: # Only calculate half of the table since diff(i,j) == diff(j,i)
                    results[i,j] = neuroNetworkDiff(self.networks[i], self.networks[j], c1, c2, c3)
        return results
        
    """
    Run a forward step through every neural network,
    and then rank all neural networks by an adjusted fitness score, defined as
    
    adj_score_i = raw_score_i / sum_j sh(delta(i,j))
    
    where delta(i,j) is the distance between NNs i and j,
    sh(x) is 1 if x > threshold, otherwise 0,
    and the sum is taken over all other NNs in the world.
    """
    def scoreNetworks(self, inputData, desiredData):
        diffMatrix = self.calcDiffNetMatrix()
        
        nnLoss = np.zeros((len(self.networks)))
        for netId in range(len(self.networks)):
            # Compute raw_score_i, using training data and a loss function
            net = self.networks[netId]
            totalNodeData = net.forwardStep(inputData)
            predicted, _, testLoss = testNet.score(totalNodeData, desiredData, squaredLoss)
            
            # Compute sum_j sh(delta(i,j))
            similarNNCount = 1
            for netId2 in range(len(self.networks)):
                if netId == netId2:
                    continue
                distanceNN = None
                if netId < netId2:
                    distanceNN = diffMatrix[netId, netId2]
                else:
                    distanceNN = diffMatrix[netId2, netId]
                if distanceNN < similarityThreshold:
                    similarNNCount += similarityPenalty
            
            nnLoss[netId] = testLoss # / similarNNCount 
            
        return nnLoss
    
    
    def scoreNetworksOnData(self, largeInputData, largeDesiredData):
        assert largeInputData.shape[0] == largeDesiredData.shape[0]
        data_n = largeInputData.shape[0]
        runningScores = np.zeros((len(self.networks)))
        
        for i in range(data_n):
            inputData = largeInputData[i]
            desiredData = largeDesiredData[i]
            indivScores = self.scoreNetworks(inputData, desiredData)
            indivScores /= data_n
            runningScores += indivScores
            
        return runningScores
    
    
    def evolveNetworks(self, largeInputData, largeDesiredData):
        testScores = self.scoreNetworksOnData(inputData, desiredData)
        testBestNNIndex = np.argsort(testScores)
        bestNetworkIds = testBestNNIndex[0:25]
        
        nextNetworks = []
        for i in range(len(bestNetworkIds)):
            mutateChildren = mutateProportion * 4 # In this hardcode, copy 4 children, mutate 2 of them
            for childIndex in range(4):
                netCopy = self.networks[bestNetworkIds[i]].copy()
                if childIndex <= mutateChildren:
                    for __ in range(iterAddMutations):
                        netCopy.mutateAddConnection()
                    for __ in range(iterSplitMutations):
                        netCopy.mutateSplitConnection()
                    for __ in range(iterRemoveMutations):
                        netCopy.mutateRemoveConnection()
                    for __ in range(iterWeightMutations):
                        netCopy.mutateChangeWeight()
                nextNetworks.append(netCopy)
        self.networks = nextNetworks
        
        return testScores

def getSortedIndexFromScore(scoresArr):
    pass
    

In [13]:
print("-----------------------------------")
print("Test #13")

inputData = np.array([1.0, 0.0, 1.0, 0.0, 1.0])
desiredData = np.array([0.0, 1.0, 0.0, 1.0, 0.0])

testWorld = World(numNetworks, numTotalNodes, numInput, numOutput)
testScores = testWorld.scoreNetworks(inputData, desiredData)

testBestNNIndex = np.argsort(testScores)

print(testScores)

print(testBestNNIndex)

-----------------------------------
Test #13
[ 1.25        1.2616996   1.24801463  1.23305456  1.27838049  1.35935759
  1.30024244  1.25        1.25        1.42642722  1.25        1.19907436
  1.59858384  1.32752429  1.1440773   1.34736333  1.31443938  1.26555263
  1.15514457  1.29559473  1.23129761  1.28378054  1.36031874  1.32446467
  1.08286316  1.42344149  1.11862748  1.39598408  1.18767989  1.25
  1.17519963  1.30444861  1.20495236  1.19006634  1.36790101  1.30672074
  1.15230478  1.41292145  1.17187223  1.29377847  1.03037384  1.38838645
  1.2972726   1.51274855  1.33255612  1.35895552  1.28458433  1.33341611
  1.24719189  1.47473815  1.246634    1.33618632  1.08705246  1.1530564
  1.2734999   1.39123395  1.25        1.4120013   1.31256426  1.25
  1.40609205  1.37538412  1.26129429  1.27664431  1.19045014  1.12744641
  0.9993099   1.25424621  1.45707771  1.22108778  1.19440768  1.3001776
  1.27497461  1.49310232  1.24752613  1.28242484  1.30746183  1.22957307
  1.383135    1.3443

C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:46: RuntimeWarning: invalid value encountered in double_scalars


In [14]:
print("-----------------------------------")
print("Test #14")

inputData = np.array([[1.0, 0.0, 1.0, 0.0, 1.0], [0.24, 0.27, 0.39, 0.56, 0.71]])
desiredData = np.array([[0.0, 1.0, 0.0, 1.0, 0.0], [0.76, 0.73, 0.61, 0.44, 0.29]])

testWorld = World(numNetworks, numTotalNodes, numInput, numOutput)
testScores = testWorld.scoreNetworksOnData(inputData, desiredData)

testBestNNIndex = np.argsort(testScores)

print(testScores)

print(testBestNNIndex)

C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:46: RuntimeWarning: invalid value encountered in double_scalars


-----------------------------------
Test #14
[ 0.63421178  0.68517844  0.63222346  0.70978918  0.69484803  0.65136748
  0.71515     0.67293029  0.68636193  0.73920794  0.71528399  0.75612991
  0.7024317   0.76733749  0.71993746  0.68675125  0.71515     0.72995427
  0.65602794  0.71515     0.8001969   0.82261424  0.91963297  0.7372353
  0.68085187  0.68906361  0.71168882  0.7125703   0.80210903  0.71515
  0.7682519   0.7396195   0.71515     0.67481628  0.62936058  0.71515
  0.71659371  0.71515     0.67434779  0.79055292  0.71515     0.80471173
  0.70596094  0.77467885  0.71497924  0.72259443  0.72764929  0.69743977
  0.70148685  0.74474287  0.70983207  0.72759585  0.70214782  0.70966304
  0.80549846  0.81693743  0.73455116  0.71515     0.84029292  0.71865034
  0.71515     0.7454974   0.69506451  0.67663969  0.72355532  0.78558504
  0.71515     0.7713506   0.75402788  0.82713088  0.67447926  0.6968321
  0.71515     0.72390053  0.76356551  0.72434308  0.71515     0.74939324
  0.77189359  

In [19]:
print("-----------------------------------")
print("Test #15")

inputData = np.array([[1.0, 0.0, 1.0, 0.0, 1.0], [0.24, 0.27, 0.39, 0.56, 0.71]])
desiredData = np.array([[0.0, 1.0, 0.0, 1.0, 0.0], [0.76, 0.73, 0.61, 0.44, 0.29]])

testWorld = World(numNetworks, numTotalNodes, numInput, numOutput)
scores = None
for _ in range(200):
    scores = testWorld.evolveNetworks(inputData, desiredData)

print(scores)

-----------------------------------
Test #15


C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:46: RuntimeWarning: invalid value encountered in double_scalars


[ 0.55903005  0.56648763  0.53753206  0.52468126  0.58896606  0.64909128
  0.54154546  0.52505114  0.5560692   0.53532716  0.70464333  0.52511119
  0.56069805  0.56097949  0.56892363  0.52511119  0.55019241  0.59906771
  0.59412711  0.52526375  0.53970553  0.56097949  0.56320805  0.52528464
  0.65950849  0.57081379  0.59202656  0.52532837  0.54559132  0.59163414
  0.54532516  0.52532837  0.59986399  0.71266522  0.67453354  0.52532837
  0.57559559  0.57430054  0.58336881  0.52532837  0.56231823  0.60566317
  0.64481513  0.52532837  0.57400263  0.53784405  0.55444017  0.52532837
  0.60969473  0.58199554  0.544165    0.52532837  0.59227248  0.58603507
  0.54626578  0.52533665  0.54451426  0.5249865   0.58046108  0.52574517
  0.63869252  0.52687404  0.56756149  0.52578523  0.56816582  0.60576864
  0.63483387  0.52578523  0.58262879  0.54540797  0.547524    0.52578523
  0.59834801  0.63557105  0.67614359  0.52578523  0.55847761  0.55600534
  0.62934329  0.52578523  0.5855393   0.66613547  0

In [20]:
testEvolvedNet = testWorld.networks[0]
testEvolvedNet.printAllEdges()

testEvolvedNet.forwardStep(np.array([1.0, 0.0, 1.0, 0.0, 1.0]))

12
10
Edge: 10 -> 8, Enabled: True, InnovID: 76565, Weight: 1
11
14
3
13
Edge: 13 -> 6, Enabled: True, InnovID: 84455, Weight: 0.9833528101639815
0
Edge: 0 -> 10, Enabled: True, InnovID: 71534, Weight: 1
4
2
Edge: 2 -> 10, Enabled: True, InnovID: 107746, Weight: 0.936904397378211
1
Edge: 1 -> 11, Enabled: True, InnovID: 114825, Weight: 0.6511814518473431


{0: 1.0,
 1: 0.0,
 2: 1.0,
 3: 0.0,
 4: 1.0,
 5: 0.5,
 6: 0.62050127094767338,
 7: 0.5,
 8: 0.70557975581302257,
 9: 0.5,
 10: 0.87401166482717241,
 11: 0.5,
 12: 0.5,
 13: 0.5,
 14: 0.5}